### Mount (montar) a Azure Data Lake Storage mediante service principal
1. Obtener el valor del client_id, tenant_id y client_secret del key Vault
2. Configurar Spark con APP/Client id , Directory/tenant Id & Secret
3. Usar el método "mount" de "utility" para montar el almacenamiento.
4. Explorar otras utilidades del sistema de archivos relacionados con el montaje (list all mounts, unmounts)


In [0]:
meta = dbutils.secrets.list(scope="movie-historyl-secret-scope")
secrets_names = [n.key for n in meta ]
print(secrets_names)

In [0]:
client_id = dbutils.secrets.get(scope="movie-historyl-secret-scope",
                    key="client2-id")
tenant_id = dbutils.secrets.get(scope="movie-historyl-secret-scope",
                    key="tenant2-id")
client_secret = dbutils.secrets.get(scope="movie-historyl-secret-scope", key="client-secret")

In [0]:

# spark.conf.set("fs.azure.account.auth.type.moviehistoryl.dfs.core.windows.net", "OAuth")
# spark.conf.set("fs.azure.account.oauth.provider.type.moviehistoryl.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
# spark.conf.set("fs.azure.account.oauth2.client.id.moviehistoryl.dfs.core.windows.net", client_id)
# spark.conf.set("fs.azure.account.oauth2.client.secret.moviehistoryl.dfs.core.windows.net", client_secret)
# spark.conf.set("fs.azure.account.oauth2.client.endpoint.moviehistoryl.dfs.core.windows.net", 
#                f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}



In [0]:
# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://demo@moviehistoryl.dfs.core.windows.net/",
  mount_point = "/mnt/moviehistoryl/demo",
  extra_configs = configs)

In [0]:
try:    display(dbutils.fs.ls("/mnt/moviehistoryl/demo"))
except Exception as e:
    print(f"Error: {e}")


In [0]:
display(spark.read
        .option("header", True)
        .option("delimiter", ";")
        .csv("/mnt/moviehistoryl/demo/movie.csv"))

In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.fs.unmount("/mnt/moviehistoryl/demo")